# Eager and Lazy Evaluation
- Eager - do tasks all at once
- Lazy - do tasks incrementally, on demand
- pros and cons to each method
- lazy advantages
    - don't need to store things until they are used
    - don't make more than you need
    - don't make things and then throw them away if not needed
    - can simulate infinite lists
- for loops can iterate over eager and lazy sequences

In [ ]:
# 'for' will loop over an existing list

x = [3,5,'sadf', .343]
for e in x:
    print(e)

In [ ]:
# range just evaluates to itself

r = range(4)
r

In [ ]:
# a range holds the values of the original args

[r.start, r.stop]

In [ ]:
# can make a list out of it

list(range(4))

In [ ]:
# for iterates over integers specified by range 

for x in range(4):
    print(x)


In [ ]:
# note: here range specifies the iteration, 
# but a million element list is never created

sum = 0
for x in range(1000000):
    sum += x
sum

In [ ]:
# here the list that range specifies IS created

rl = list(range(4))

print(rl)

for x in rl:
    print(x)

# Iterator Protocol
- there is a general protocol for iterating over objects
- use 'iter' function to get an iterator from an object
    - not all objects have iterators - for example, int and float don't
- the 'iterator' may be the same object, or a different one
- some objects allow multiple iterators simultaneously
- call 'next' function repeatedly, to get the elements of the iteration
- when all elements have been produced, iterator will raise a 'StopIteration' error each
time 'next' is called
- 'StopIteration' implies the iterator is 'exhausted' - discard it.
- why raise an error at the end of the iteration???
- for loops use iterator protocol

In [ ]:
x = [1,4]
xi = iter(x)
xi

In [ ]:
# 1st value

next(xi)

In [ ]:
# 2nd value

next(xi)

In [ ]:
# done

next(xi)

In [ ]:
# 'range' each iterator is a new obj - can have any number of them

r = range(2)
ri = iter(r)
ri2 = iter(r)

[r, ri, ri2, ri is r, ri is ri2]

In [ ]:
next(ri)

In [ ]:
[next(ri), next(ri2)]

In [ ]:
# now ri is ahead of ri2, so it finishes first
next(ri)

In [ ]:
# one val left for ri2

next(ri2)

In [ ]:
# now ri2 is done

next(ri2)

# Generator Function
- one way to define an iterator
- a generator is defined by using a 'yield' statement inside a 'def'
- executing the function returns the iterator
- falling off the end of the function, or executing a 'return' statement, will terminate the generator.
- once a generator terminates, it is 'exhausted', and can not be used again
- calling 'next' on a generator will cause the generator to execute until it 
hits a 'yield' statement. The arg supplied to 'yield' will be returned by 'next'. The next time 'next' is called on the generator, the generator will resume executing on the statement following the yield. 
- all local variable values are preserved between between 'next' calls to the generator

In [ ]:
# executing this function will return a generator

def gf(n):
    for j in range(n):
        yield(j)

In [ ]:
# 'list' will run generator until it is exhausted.

list(gf(5))

In [ ]:
# or can use returned generator explicitly via iteration protocol

g = gf(2)
g

In [ ]:
next(g)

In [ ]:
next(g)

In [ ]:
# generator is finished - discard it

next(g)

In [ ]:
# iterate over generator directly

[j+10 for j in gf(3)]

# A generator can represent an infinite sequence (sort of)
- can't make a list of all the even integers
- but in some sense can represent that list with a generator

In [ ]:
def infinite(start, incr):
    e = start
    # this generator will never terminate
    while True:
        yield(e)
        e += incr

In [ ]:
# eg represents the positive even numbers

eg = infinite(2,2)
[next(eg) for j in range(5)]

In [ ]:
# a generator can use another generator

def evenPowersOf2():
    eg = infinite(2,2)
    while True:
        e = next(eg)
        yield 2**e

ep2 = evenPowersOf2()
[next(ep2) for j in range(5)]

In [ ]:
import operator

# add series

eg = infinite(2,2)
g5 = infinite(5,5)

# generators can use other generators

def opgen(op, g1, g2):
    while True: 
        e1 = next(g1)
        e2 = next(g2)
        yield op(e1,e2)
        
og = opgen(operator.add, eg, g5)

[next(og) for j in range(5)]


In [ ]:
# subtract series

eg = infinite(2,2)
g5 = infinite(5,5)
og = opgen(operator.sub, eg, g5)

[next(og) for j in range(5)]

# Yields do not have to be inside a loop
- fibonacci series is 1,1,2,3,5,8...
- f(0) = 1
- f(1) = 1
- f(n) = f(n-1) + f(n-2)

In [ ]:
def fibonacci():
    # easy way to handle the first two ones
    yield(1)
    yield(1)
    last = 1
    last2 = 1
    while True:
        sum = last + last2
        yield(sum)
        last2 = last
        last = sum

f = fibonacci()

for j in range(10):
    print( next(f))


# Modifying a Running Generator
- can change generator state at any time

In [ ]:
def counter(maximum):
    cnt = 0
    while cnt < maximum:
        # peculiar syntax
        val = (yield cnt)
        # If value provided, change counter
        if val is not None:
            cnt = val
        else:
            cnt += 1


In [ ]:
c = counter(1000)
[next(c) for j in range(10)]

In [ ]:
# change the 'cnt' variable that the generator saves 
# '(yield cnt)' in generator will return 300

c.send(300)

# generator continues from new value
[next(c) for j in range(10)]

In [ ]:
# the generator is nowhere near done, but we can terminate it

c.close()

In [ ]:
# the generator is exhausted now

next(c)

# Generator Expression
- an expression that evaluates to a generator
- looks like a list comprehension, but with outer '()' instead of '[]'

In [ ]:
def ge(n):
    # can't return a 'def'
    return ( j**2 for j in range(2, n) if j != 3)

g = ge(8)

In [ ]:
# pick first two manually

next(g)

In [ ]:
# skipped j == 3

next(g)

In [ ]:
# for gets the rest

for j in g:
    print(j)

# suppose want to sum 100,000 squares...

In [ ]:
# could do

mil = 1000**2

sq = [x**2 for x in range(mil)]
sum(sq)


In [ ]:
# or 

total = 0 
for x in range(mil):
    total += x**2
total

In [ ]:
# could use a generator

# which way is better?

sum(x**2 for x in range(mil))

# A generator will finish if it calls a generator that finishes

In [ ]:
def g(n):
    for j in range(n):
        yield j

def g2(n):
    gen = g(n)
    while True:
        yield next(gen)


In [ ]:
list(g2(3))

In [ ]:
# generate chars

def chars(s):
    for c in s:
        yield c
        
cs = chars('larry')
for c in cs:
    print(c)
    

In [ ]:
# 'yield from' will yield everything from its generator argument

def gfrom(g):
    yield from g
    
gs = gfrom(chars('larry'))

for c in gs:
    print(c)